# Play around with generating text from an LLM

First, let's check if we have a GPU.

In [ ]:
!pip install sentencepiece

In [ ]:
import torch
if torch.cuda.is_available():
    print('Using GPU, device name:', torch.cuda.get_device_name(0))
    device = torch.device('cuda')
else:
    print('No GPU found, using CPU instead.') 
    device = torch.device('cpu')

Let's change the path for the Huggingface cache where it stores the models it downloads.

In [ ]:
import os
os.environ["HF_HOME"] = "/scratch/project_462000584/huggingface"

We'll create a helper function for printing the generated texts

In [ ]:
def print_output(output):
    for item in output:
        text = item['generated_text']
        text = text.replace("<br />", "\n")
        print('-', text)
        print()

We'll create a generator object using the Huggingface Transformers pipeline.

Note that you have to select what model to use from the Huggingface model repository: https://huggingface.co/models

You can change the model by modifying the `model_name` variable. For example "distilgpt2" is a light and fast model similar to GPT-2.


In [ ]:
from transformers import pipeline

#model_name = 'distilgpt2'
#model_name = 'microsoft/phi-2'
model_name = "NousResearch/Hermes-2-Pro-Mistral-7B"

generator = pipeline("text-generation", model=model_name, device=device)

Finally, let's try generating an output based on a prompt.

In [ ]:
output = generator("The AI Sauna")
print_output(output)

## Experiment with the generation strategy

You can play with the text generation if you wish. Text generation strategies are discussed here: https://huggingface.co/docs/transformers/generation_strategies

Note that we are here using the easy-to-use `TextGenerationPipeline` and its `generator()` function, but the link discusses the `model.generate()` method. The same parameters can be used, though, the pipeline just takes care of some of the pre- and post-processing.

In particular these parameters of the `generator()` function might be interesting:

- `max_new_tokens`: the maximum number of tokens to generate
- `num_beams`: activate Beam search by setting this > 1
- `do_sample`: activate multinomial sampling if set to True
- `num_return_sequences`: the number of candidate sentences to return (available only for beam search and sampling)

Here is a nice blog post explaining in more detail about the different generation strategies: https://huggingface.co/blog/how-to-generate

In [ ]:
output = generator("What is the AI Sauna event about?", num_return_sequences=1, max_new_tokens=100)
print_output(output)

# Generate images with Stable Diffusion XL

In [ ]:
!pip install diffusers

In [ ]:
import torch
from diffusers import StableDiffusionXLPipeline

pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16
)
pipe = pipe.to(device)

In [ ]:
prompt = "A photo from the AI sauna hackathon"
image = pipe(prompt).images[0]
image

In [ ]:
image.save("generated_image.jpg")